In [1]:
import os
import random as rnd
import pandas as pd 

import pgutils as pg
import pgmath as pm
import algorithm as al
import numpy as np

In [2]:
root_dir = r'C:\Users\eleon\PycharmProjects\SM_Exam\data\yes_small'
#root_dir = './data/yes_small'

In [3]:
#test_data=[]
with open(os.path.join(root_dir, 'test.txt')) as f:
#with open(root_dir + "\\test.txt", "r") as f:

    test_data = f.readlines()
    
test_dataset = pg.data_to_list(test_data[2:])

In [4]:
#train_data=[]
with open(os.path.join(root_dir,"train.txt"), "r") as f:
    train_data = f.readlines()
    
#train_dataset = pg.data_to_list(train_data[2:])

In [5]:
song_hash = pd.read_csv(os.path.join(root_dir,"song_hash.txt"), sep="\t", header=None)


In [6]:
# setting parameters
dimension = 2#rnd.choice([2, 5,10,25,50,100])
# regularization parameter set by cross validation
lam = rnd.choice([0.0001, 0.001, 0.01, 0.1, 1, 10, 20, 50, 100, 500, 1000])
# regularization parameter for dual point model
nu = rnd.choice([0, lam])
# threshold for landmark dimension
r = 0.2
# number of landmark
n_landmarks = 3#50
# num iteration 100 o 200
n_iter = 60 #rnd.choice([100, 200])
# tau predefined learning rate
tau = 0.5

In [7]:
toy_dataset=[[0,6,3,2,1,4,2],[2,0,6,5,6,3,0],[0,5,2,1,6],[5,6,2],[6,5,2,3,5,6,2]]

songs = 7
transition_matrix = pg.transition_count(songs, toy_dataset)
position = np.random.rand(songs, dimension)
num_transition = np.sum(transition_matrix)
params = pm.AlgParams(lam, nu, tau, num_transition, n_landmarks, r, dimension, n_iter)

In [8]:
X = al.single_point_algorithm(songs, transition_matrix, params)

0
1


In [9]:
dummy_landmarks = [[i for i in range(songs)] for j in range(songs)]
d2 = pm.Distances.delta(X)
prob_matrix = np.exp(-d2)/pm.Distances.zeta(d2,X,dummy_landmarks)
cum_sum = np.sum(prob_matrix, axis=1)
prob_matrix = prob_matrix / cum_sum[:,np.newaxis]

In [10]:
print('cum',cum_sum)
print('sum norm', np.sum(prob_matrix, axis=1))
#print('sum row ',np.sum(prob_matrix/cum_sum[0], axis=0))
print(np.sum(prob_matrix[1]))

cum [0.98411027 0.82799826 0.77853495 0.80357572 0.96692714 0.87388325
 0.98741885]
sum norm [1. 1. 1. 1. 1. 1. 1.]
0.9999999999999998


In [11]:
print(prob_matrix)
print(np.sum(prob_matrix, axis=0))

[[0.18517481 0.10331874 0.12013696 0.10792529 0.17160203 0.15614964
  0.15569253]
 [0.09713087 0.27824824 0.1254254  0.20512436 0.09839727 0.08683349
  0.10884037]
 [0.11187989 0.12424609 0.31771508 0.10030235 0.10441071 0.13138931
  0.11005657]
 [0.09892294 0.1999921  0.09872099 0.3029996  0.10428702 0.08427472
  0.11080263]
 [0.17191616 0.10485742 0.11232177 0.1139858  0.19146424 0.14809756
  0.15735706]
 [0.15694156 0.09283379 0.14180177 0.09241033 0.14857666 0.23364992
  0.13378597]
 [0.15954008 0.11863529 0.12109958 0.12387349 0.16095112 0.1364004
  0.17950005]]
[0.98150631 1.02213168 1.03722155 1.04662122 0.97968904 0.97679504
 0.95603517]


In [12]:
toy_test=[[0,4,5,3,6],[4,3,2,6],[0,2,1,0,5]]
# evaluate test performance using the average log-likelihood as metric
# it is defined as log(Pr(d_test))/n_test)
# n_test number of transition in the test set
songs = 7 #len(song_hash)
n_test = np.sum(pg.transition_count(songs,toy_test))
print(n_test)

11.0


In [13]:
evaluation = pg.log_like(toy_test,prob_matrix) /n_test
print(evaluation)
#result = sum( sum(dimension.D[s, i.index()] ) for i in range(len(test_dataset)))  

-2.121921175101047


In [14]:
#generate a playlist
pg.playlist_generator(4,5,song_hash,prob_matrix)





Song in the playlist:  ['Down On Me (w\\/ 50 Cent)', 'Hot Toddy (w\\/ Jay-Z & Ester Dean)', 'Aston Martin Music (w\\/ Drake & Chrisette Michelle)', 'Aston Martin Music (w\\/ Drake & Chrisette Michelle)', 'Hot Toddy (w\\/ Jay-Z & Ester Dean)']
End state after  4  days:  Hot Toddy (w\/ Jay-Z & Ester Dean)
Probability of the possible sequence of states:  0.001054831351205396
